# 1. Shortest Path in a Grid with Obstacles Elimination

You are given an m x n integer matrix grid where each cell is either 0 (empty) or 1 (obstacle). You can move up, down, left, or right from and to an empty cell in one step.

Return the minimum number of steps to walk from the upper left corner (0, 0) to the lower right corner (m - 1, n - 1) given that you can eliminate at most k obstacles. If it is not possible to find such walk return -1.

![img](https://assets.leetcode.com/uploads/2021/09/30/short1-grid.jpg)

Example 1:
```
Input: grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]], k = 1
Output: 6
Explanation: 
The shortest path without eliminating any obstacle is 10.
The shortest path with one obstacle elimination at position (3,2) is 6. Such path is (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (3,2) -> (4,2).
```

Overview

Like many grid search problems where the goal is to find the shortest path, the key to solve this problem is to apply the Breadth-First Search algorithm, as opposed to the Depth-First Search (DFS) algorithm. In this article, we will start with a classic BFS solution. Then on top of BFS, we will introduce a heuristic (greedy) strategy to speed up the algorithm, which eventually transforms our classic BFS algorithm into another classic algorithm called the A* search algorithm.

Approach 1: BFS (Breadth-First Search)

Intuition

In this problem, we must traverse the grid to reach a target cell, while the grid contains some obstacles. If the problem ends here, one could probably tell that this is a classic grid search problem, e.g. the problem of 1730. Shortest Path to Get Food fits the bill exactly.

However, the particularity of this problem is that one can eliminate obstacles to a certain extent. This constraint complicates our problem. First of all, if there were no limit on how many obstacles we can eliminate, then the shortest distance to reach the target cell would be the Manhattan distance between the starting cell and the target cell. Likewise, if the quota to eliminate the obstacles is greater than the Manhattan distance, then the shortest distance is guaranteed to be the Manhattan distance. However, we do have a limit on the number of obstacles that we can eliminate along the way. As a result, rather than taking a straightforward path to reach the target, we have to take some detour in certain cases, which implies that we need to explore all possible directions while respecting the constraint.

By exploring, we refer to the BFS strategy, rather than DFS. The BFS algorithm works like detecting an object with sonar. A sound wave propagages in all directions with equal speed. At any given moment, all the objects that the sound wave reaches have the same distance to the source of the sound. On the other hand, as soon as the sound wave reaches the object, the path is guaranteed to be the shortest, since the distance is proportional to the time, the more time it takes, the longer the distance is.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs.png)

Given the above intuition of the BFS algorithm, we can rest assured that as soon as we reach the target, the path that leads to the target is the shortest. This is also the rationale for why we should employ the BFS strategy rather than DFS.

Algorithm

In the canonical implementation of the BFS algorithm, we would employ a queue data structure to maintain the order of exploration. Each element in the queue normally contains two pieces of information: the current position and the distance traveled so far from the starting point.

However, in our problem here, we need another piece of information, which is the remaining quota that we can use to eliminate the obstacles.

Together with the coordinates, the obstacle elimination quota uniquely constitutes a state during our BFS exploration. For example, in the following graph, we demonstrate two different paths between the start cell and an intermediate cell.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs_two_paths.png)

Without the obstacle elimination quota, we would only visit the intermediate cell once, while only one of the two paths can lead us to the target, since in one of the paths we don't have sufficient quota to get through. Therefore, it is critical to keep the quota information, so that we can revisit the same cell with different quotas.

Implementation

There are several ways to implement a BFS algorithm. We provide a template to do so in the Queue and Stack Explore Card.

In this section, we provide some sample implementations together with the tweak we mentioned in the above section.

We can break down the algorithm into the following steps:

1. The main body of the BFS algorithm consists of a loop around a queue, as well as a set called seen which keeps track of all the states visited along the way so that we don't visit the same state twice. A state refers to a unique combination of coordinates and the remaining quota.

2. At each iteration of the loop, we pop out one element from the queue. The element contains the distance from the starting point as well as the current state which includes the current coordinate and the remaining quotas to eliminate obstacles.

3. Within the same iteration, we evaluate the next moves starting from the popped element. Each move results in a new state and the state is valid if it is within the grid boundaries and has not been visited before. Each valid state is pushed into the queue for later iterations. Meanwhile, we also need to mark the state as visited by adding the state to the seen states set.

4. At any iteration, if we have reached our target, we can return immediately from the loop.

Note: before running the BFS traversal, we also perform a quick check to see if we have sufficient quotas to take the most direct path from start to finish regardless of the number of obstacles. If so, the shortest distance to reach the target is the Manhattan distance between the starting cell and the target cell.


In [3]:
from collections import deque
class Solution:
    def shortestPath(self, grid, k):
        rows, cols = len(grid), len(grid[0])

        target = (rows-1, cols-1)

        if k >= rows+cols-2:
            return rows+cols-2


        # (row, col, remaining quota)
        state = (0, 0, k)

        queue = deque([(0, state)])

        seen = set([state])

        while queue:
            steps, (row, col, k) = queue.popleft()

            if (row, col) == target:
                return steps
            
            for new_row, new_col in [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]:
                if (0<=new_row<rows) and (0<=new_col<cols):
                    new_elimination = k - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_elimination)

                    if new_elimination>=0 and new_state not in seen:
                        seen.add(new_state)
                        queue.append((steps+1, new_state))
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))





6


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

Time Complexity: O(N \cdot K)O(N⋅K)

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And for each cell, at most, it will be visited KK times, with different quotas of obstacle elimination.

Thus, the overall time complexity of the algorithm is O(N \cdot K)O(N⋅K).

Space Complexity: O(N \cdot K)O(N⋅K)

We used a queue to maintain the order of visited states. In the worst case, the queue will contain the majority of the possible states that we need to visit, which in total is N \cdot KN⋅K as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the visited states along the way. Same as the queue, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).

Approach 2: A* (A Star) Algorithm

Intuition

In the above BFS approach, one might notice that when at any specific position, we would systematically explore the surrounding neighbors in all four directions, due to the nature of BFS.

However, the action might seem conterintuitive or sub-optimal. Since the destination is located in the lower-right corner of the grid, in order to find the shortest path, the optimal directions to explore should be either right or down, rather than left or up.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_next_steps.png)

As depicted in the above image, the optimal steps to explore should be the ones in green (right and down), rather than the ones in orange.

Therefore, one idea to improve our BFS approach is to prioritize exploring the most promising directions at each step. Through prioritization, we can speed up the algorithm by reducing the time spent exploring less promising paths.

This idea leads us to the A* search algorithm, which is yet another classic path finding algorithm that uses a heuristic.

Note: we cannot exclude (or prune) those less promising directions, otherwise we might miss the correct path because sometimes we have to take a detour in order to reach the destination.

Algorithm

A* (pronounced as A star) is also known as an informed search algorithm or best-first search. Because at each step of exploration, it makes the best and informed decision on the next steps, i.e. it prioritizes the steps that are the most promising. Specifically, this prioritization strategy can be expressed as A* selects a path that minimizes the following function:f(n) = g(n) + h(n)f(n)=g(n)+h(n)

nn: a specific step during the exploration.

g(n)g(n): the cost to reach the step nn. Here, the cost refers to the distance traveled so far to the step nn.

h(n)h(n): a heuristic estimation on the cost to reach the destination from the step nn. Here, the cost refers to the distance ahead.

f(n)f(n): the estimated total cost to reach the destination if one takes the step nn.

With the defined function, A* algorithm has a deterministic way to evaluate each potential step, and then make what it believes to be the best decision at each step.

The problem boils down to defining the above functions for our scenario, in order to apply the A* algorithm. Specifically, g(n)g(n) would be the number of steps that one has taken to reach nn. And h(n)h(n) would be the Manhattan distance from nn to the destination, which is the shortest path to reach the destination.

The most important property of our heuristic h(n)h(n) function is that the function should be admissible, i.e. it never overestimates the cost. Otherwise, it could not guarantee that the path we find is the shortest one.

To understand the admissible property, let us take a metaphor. In a football tournament, we want to select the best team in the end. If we overestimate the incompetence of a team, i.e. we downplay the potential of the team, we might prematurely disqualify or ignore the team. As a result, we may predict that a not-so-good team will win the championship, while in reality, the team we disqualified or ignored happens to be the best team.

Implementation

The A* algorithm provides a more optimized path selection strategy, on top of the BFS approach. Therefore, we can implement the A* algorithm while keeping the bulk of our previous BFS approach intact. We will still use a queue to keep track of the order of visits. And we will still use a set to keep track of the visited states so that we do not revisit any previously explored paths.

Additionally, here are the modifications that we will make:

* Rather than using a normal queue, we use a priority queue to store the order of visits. The order of visits is based on the estimated total cost function f(n)f(n) that we defined. With the priority queue, the step that has potentially the lowest cost will be visited first.

* For each element in the queue, we add one more piece of information which is the estimated total cost to reach the destination at each step. This estimation will be used to prioritize each potential next step.

* We add another heuristic condition that allows us to determine the length of the shortest path without exploration. The condition is as follows:

    * At any step, if the remaining quota to eliminate the obstacles is larger than the length of the estimated shortest path (i.e. manhattan distance between the current step and the destination), then the length of the remaining path is the manhattan distance.

    * The condition can also be interpreted as if we have sufficient capacity to remove any obstacle along the way, we will simply take the shortest path to reach the destination, without the need for further exploration.

    * We apply the condition at the beginning of each iteration of the loop.


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

* Time Complexity: O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K))

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And each cell can be visited at most KK times, with different quotas of obstacle elimination. Therefore, the total number of visits would be N \cdot KN⋅K.

For each visit, we perform one push and one pop operation in the priority queue, which takes O\big(\log{(N \cdot K)} \big)O(log(N⋅K)) time.

Thus, the overall time complexity of the algorithm is O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K)).

Although the upper bound for the time complexity of the this algorithm is higher than the previous BFS approach, on average, the A* algorithm will outperform the previous BFS approach when there exists any relatively direct path from the source to the target.

* Space Complexity: O(N \cdot K)O(N⋅K)

We use a queue to maintain the order of visited states. In the worst case, the queue could contain the majority of the possible states that we must visit, which in total is N \cdot KN⋅K, as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the states we visited along the way. Again, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).






In [6]:
import heapq
class Solution:
    def shortestPath(self, grid, k):

        rows, cols = len(grid), len(grid[0])
        target = (rows - 1, cols - 1)

        def manhattan_distance(row, col):
            return target[0] - row + target[1] - col

        # (row, col, remaining_elimination)
        state = (0, 0, k)

        # (estimation, steps, state)
        # h(n) = manhattan distance,  g(n) = 0
        queue = [(manhattan_distance(0, 0), 0, state)]
        seen = set([state])

        while queue:
            estimation, steps, (row, col, remain_eliminations) = heapq.heappop(queue)

            # we can reach the target in the shortest path (manhattan distance),
            #   even if the remaining steps are all obstacles
            remain_min_distance = estimation - steps
            if remain_min_distance <= remain_eliminations:
                return estimation

            # explore the four directions in the next step
            for new_row, new_col in [(row, col + 1), (row + 1, col), (row, col - 1), (row - 1, col)]:
                # if (new_row, new_col) is within the grid boundaries
                if (0 <= new_row < rows) and (0 <= new_col < cols):
                    new_eliminations = remain_eliminations - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_eliminations)

                    # if the next direction is worth exploring
                    if new_eliminations >= 0 and new_state not in seen:
                        seen.add(new_state)
                        new_estimation = manhattan_distance(new_row, new_col) + steps + 1
                        heapq.heappush(queue, (new_estimation, steps + 1, new_state))

        # did not reach the target
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))

6


# 2. Step-By-Step Directions From a Binary Tree Node to Another

You are given the root of a binary tree with n nodes. Each node is uniquely assigned a value from 1 to n. You are also given an integer startValue representing the value of the start node s, and a different integer destValue representing the value of the destination node t.

Find the shortest path starting from node s and ending at node t. Generate step-by-step directions of such path as a string consisting of only the uppercase letters 'L', 'R', and 'U'. Each letter indicates a specific direction:

* 'L' means to go from a node to its left child node.
* 'R' means to go from a node to its right child node.
* 'U' means to go from a node to its parent node.

Return the step-by-step directions of the shortest path from node s to node t.

Example 1:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg1.png)
```
Input: root = [5,1,2,3,null,6,4], startValue = 3, destValue = 6
Output: "UURL"
Explanation: The shortest path is: 3 → 1 → 5 → 2 → 6.
```

Example 2:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg2.png)
```
Input: root = [2,1], startValue = 2, destValue = 1
Output: "L"
Explanation: The shortest path is: 2 → 1.
```

## Approach 1

* Build directions for both start and destination from the root.
    * Say we get "LLRRL" and "LRR".

* Remove common prefix path.
    * We remove "L", and now start direction is "LRRL", and destination - "RR"

* Replace all steps in the start direction to "U" and add destination direction.
    * The result is "UUUU" + "RR".

In [9]:
class Node:
    def __init__(self, value):
        self.val = value
        self.left, self.right = None, None

class Solution:
    def getDirections(self, root, startValue, destValue):
        def find(n, value, path):
            if n.val == value:
                return True
            if n.left and find(n.left, value, path):
                path+= "L"
            elif n.right and find(n.right, value, path):
                path+= "R"
            return path 
        s, d = [], []
        s = find(root, startValue, s)
        d = find(root, destValue, d)

        while len(s) and len(d) and s[-1]==d[-1]:
            s.pop()
            d.pop()
        return "".join("U" * len(s)) + "".join(reversed(d))

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))


UURL


## Approach 2: LCA


In [10]:
class Solution:
    def getDirections(self, root, startValue, destValue):
        
        def lca(node): 
            """Return lowest common ancestor of start and dest nodes."""
            if not node or node.val in (startValue , destValue): return node 
            left, right = lca(node.left), lca(node.right)
            return node if left and right else left or right
        
        root = lca(root) # only this sub-tree matters
        
        ps = pd = ""
        stack = [(root, "")]
        while stack: 
            node, path = stack.pop()
            if node.val == startValue: ps = path 
            if node.val == destValue: pd = path
            if node.left: stack.append((node.left, path + "L"))
            if node.right: stack.append((node.right, path + "R"))
        return "U"*len(ps) + pd

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))

UURL


# 3. Race Car

Your car starts at position 0 and speed +1 on an infinite number line. Your car can go into negative positions. Your car drives automatically according to a sequence of instructions 'A' (accelerate) and 'R' (reverse):

* When you get an instruction 'A', your car does the following:
```python
position += speed
speed *= 2
```
* When you get an instruction 'R', your car does the following:
```python
If your speed is positive then speed = -1
otherwise speed = 1
```
Your position stays the same.

For example, after commands "AAR", your car goes to positions 0 --> 1 --> 3 --> 3, and your speed goes to 1 --> 2 --> 4 --> -1.

Given a target position target, return the length of the shortest sequence of instructions to get there.

 
```
Example 1:

Input: target = 3
Output: 2
Explanation: 
The shortest instruction sequence is "AA".
Your position goes from 0 --> 1 --> 3.
Example 2:

Input: target = 6
Output: 5
Explanation: 
The shortest instruction sequence is "AAARA".
Your position goes from 0 --> 1 --> 3 --> 7 --> 7 --> 6.
```

I -- BFS solution

Well, the BFS solution is straightforward: we can keep track of all the possible positions of the racecar after n instructions (n = 0, 1, 2, 3, 4, ...) and return the smallest n such that the target position is reached. Naive BFS will run at O(2^n) since for each position we have two choices: either accelerate or reverse. Further observations reveal that there may be overlapping among intermediate states so we need to memorize visited states (each state is characterized by two integers: car position and car speed). However, the total number of unique states still blows up for large target positions (because the position and speed can grow unbounded), so we need further pruning of the search space.

II -- DP solution

DP solution works by noting that after each reverse, the car's speed returns to 1 (the sign can be interpreted as the direction of the speed). So we can redefine the problem in terms of the position of the car while leave out the speed: let T(i) be the length of the shortest instructions to move the car from position 0 to position i, with initail speed of 1 and its direction pointing towards position i. Then our original problem will be T(target), and the base case is T(0) = 0. Next we need to figure out the recurrence relations for T(i).

Note that to apply the definition of T(i) to subproblems, the car has to start with speed of 1, which implies we can only apply T(i) right after the reverse instruction. Also we need to make sure the direction of the initial speed when applying T(i) is pointing towards the final target position.

However, we don't really know how many accelerate instructions there should be before the reverse instruction, so theoretically we need to try all possible cases: zero A, one A, two A, three A, ... and so on. For each case, we can obtain the position of the car right before the reverse instruction, which will be denoted as j = 2^m - 1, with m the number of A's. Then depending on the relation between i and j, there will be three cases:

j < i: the reverse instruction is issued before the car reaches i. In this case, we cannot apply the definition of T(i) to the subproblems directly, because even though the speed of the car returns to 1, its direction is pointing away from the target position (in this case position i). So we have to wait until the second reverse instruction is issued. Again, we don't really know how many accelerate instructions there should be in between these two reverse instructions, so we will try each of the cases: zero A, one A, two A, three A, ..., etc. Assume the number of A is q, then the car will end up at position j - p right before the second reverse instruction, where p = 2^q - 1. Then after the second reverse instruction, our car will start from position j - p with speed of 1 and its direction pointing towards our target position i. Since we want the length of the total instruction sequence to be minimized, we certainly wish to use minimum number of instructions to move the car from j - p to i, which by definition will be given by T(i-(j-p)) (note that in the definition of T(i), we move the car from position 0 to position i. If the start position is not 0, we need to shift both the start and target positions so that the start position is aligned with 0). So in summary, for this case, the total length of the instruction will be given by: m + 1 + q + 1 + T(i-(j-p)), where m is the number of A before the first R, q is the number of A before the second R, the two 1's correspond to the two R's, and lastly T(i-(j-p)) is the length of instructions moving the car from position j - p to the target position i.

j == i: the target position is reached without any reverse instructions. For this case, the total length of the instruction will be given by m.

j > i: the reverse instruction is issued after the car goes beyond i. In this case, we don't need to wait for a second reverse instruction, because after the first reverse instruction, the car's speed returns to 1 and its direction will be pointing towards our target position i. So we can apply the definition of T(i) directly to the subproblem, which will be T(j-i). Note that not only do we need to shift the start and target positions, but also need to swap them as well as the directions. So for this case, the total length of the instructions will be given by m + 1 + T(j-i).

Our final answer for T(i) will be the minimum of the above three cases.

**Time and Space Complexity:**
Both the top-down DP and bottom-up DP run at O(target * (log(target))^2) with O(target) space. However, the top-down DP may be slightly more efficient as it may skip some of the intermediate cases that must be computed explicitly for the bottom-up DP. Though the nominal time complexity are the same, both DP solutions will be much more efficient in practice compared to the BFS solution, which has to deal with (position, speed) pairs and their keys for hashing, etc.

In [14]:
# bfs 
import collections
class Solution:
    def racecar(self, target: int) -> int:
        
        #1. Initialize double ended queue as 0 moves, 0 position, +1 velocity
        queue = collections.deque([(0, 0, 1)])
        while queue:
            
            # (moves) moves, (pos) position, (vel) velocity)
            moves, pos, vel = queue.popleft()

            if pos == target:
                return moves
            
            #2. Always consider moving the car in the direction it is already going
            queue.append((moves + 1, pos + vel, 2 * vel))
            
            #3. Only consider changing the direction of the car if one of the following conditions is true
            #   i.  The car is driving away from the target.
            #   ii. The car will pass the target in the next move.  
            if (pos + vel > target and vel > 0) or (pos + vel < target and vel < 0):
                queue.append((moves + 1, pos, -vel / abs(vel)))

# Driver Code
if __name__ == "__main__":
    target = 3
    print(Solution().racecar(target))   

2


In [12]:
# dp solution
class Solution:
    def __init__(self):
        self.dp = {0: 0}
    def racecar(self, t):
        if t in self.dp:
            return self.dp[t]
        n = t.bit_length()  
        print(f"n = {n}")
        if 2**n - 1 == t:
            self.dp[t] = n
        else:
            self.dp[t] = self.racecar(2**n - 1 - t) + n + 1
            for m in range(n - 1):
                self.dp[t] = min(self.dp[t], self.racecar(t - 2**(n - 1) + 2**m) + n + m + 1)
        return self.dp[t]

# Driver Code
if __name__ == "__main__":
    t = 3
    print(Solution().racecar(t))


n = 2
2


# 4. Find Leaves of Binary Tree

Given the root of a binary tree, collect a tree's nodes as if you were doing this:

Collect all the leaf nodes.
Remove all the leaf nodes.
Repeat until the tree is empty.
 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/03/16/remleaves-tree.jpg)
```
Input: root = [1,2,3,4,5]
Output: [[4,5,3],[2],[1]]
Explanation:
[[3,5,4],[2],[1]] and [[3,4,5],[2],[1]] are also considered correct answers since per each level it does not matter the order on which elements are returned.
Example 2:

Input: root = [1]
Output: [[1]]
```

Approach 1: DFS (Depth-First Search) with sorting

Intuition

The order in which the elements (nodes) will be collected in the final answer depends on the "height" of these nodes. The height of a node is the number of edges from the node to the deepest leaf. The nodes that are located in the ith height will be appear in the ith collection in the final answer. For any given node in the binary tree, the height is obtained by adding 1 to the maximum height of any children. Formally, for a given node of the binary tree \text{root}root, it's height can be represented as

$$\text{height(root)} = \text{1} + \text{max(height(root.left), height(root.right))}$$

Where $\text{root.left}$ and $\text{root.right}$ are left and right children of the root respectively

Algorithm

In our first approach, we'll simply traverse the tree recursively in a depth first search manner using the function int getHeight(node), which will return the height of the given node in the binary tree. Since height of any node depends on the height of it's children node, hence we traverse the tree in a post-order manner (i.e. height of the childrens are calculated first before calculating the height of the given node). Additionally, whenever we encounter a null node, we simply return -1 as it's height.

Next, we'll store the pair (height, val) for all the nodes which will be sorted later to obtain the final answer. The sorting will be done in increasing order considering the height first and then the val. Hence we'll obtain all the pairs in the increasing order of their height in the given binary tree.

Complexity Analysis

Time Complexity: Assuming NN is the total number of nodes in the binary tree, traversing the tree takes O(N)O(N) time. Sorting all the pairs based on their height takes O(N \log N)O(NlogN) time. Hence overall time complexity of this approach is O(N \log N)O(NlogN)

Space Complexity: O(N)O(N), the space used by pairs. solution also requires O(N)O(N) space however the output does not count towards the space complexity.


In [25]:
class Solution:
    def __init__(self):
        self.pairs = []

    def getHeight(self, root):
        if not root:
            return -1
        leftHeight = self.getHeight(root.left)
        rightHeight = self.getHeight(root.right)

        currHeight = max(leftHeight, rightHeight) + 1

        self.pairs.append((currHeight, root.val))
        return currHeight
    
    def findLeaves(self, root):
        self.getHeight(root)
        self.pairs.sort(key=lambda x: x[0])
        # print(self.pairs)
        n = len(self.pairs)
        height = 0
        i=0
        solution = []
        while i<n:
            nums = []
            while i<n and self.pairs[i][0] == height:
                nums.append(self.pairs[i][1])
                i += 1
            solution.append(nums)
            # print(solution)
            height += 1
        return solution

# Driver Code
if __name__ == "__main__":
    root = Node(1)
    root.left = Node(2)
    root.right = Node(3)
    root.left.left = Node(4)
    root.left.right = Node(5)
    print(Solution().findLeaves(root))


[[4, 5, 3], [2], [1]]


# 5.  Remove All Ones With Row and Column Flips

You are given an m x n binary matrix grid.

In one operation, you can choose any row or column and flip each value in that row or column (i.e., changing all 0's to 1's, and all 1's to 0's).

Return true if it is possible to remove all 1's from grid using any number of operations or false otherwise.

Example 1:
![img](https://assets.leetcode.com/uploads/2022/01/03/image-20220103191300-1.png)
```
Input: grid = [[0,1,0],[1,0,1],[0,1,0]]
Output: true
Explanation: One possible way to remove all 1's from grid is to:
- Flip the middle row
- Flip the middle column
```

Explanation
```
I honestly don't know how to categorize this problem. It seems like a Math problem to me. Once you understand the logic, the implementation is simple.
Basically the "pattern" of each row should be the same, by pattern, I mean following:
001100 and 001100 are the same pattern
001100 and 110011 (the invert of original) are the same pattern
Only in above situation, one matrix can be converted to all zero
Intuition?
Believe it or not, I draw a couple examples to test it out and suddenly it becomes obvious
I guess it's good habit to get your hands dirty :)
```



In [31]:
class Solution:
    def removeOnes(self, grid):
        r1, r1_invert = grid[0], [1-val for val in grid[0]]
        for i in range(1, len(grid)):
            if grid[i] != r1 and grid[i] != r1_invert:
                return False
        return True

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True


In [30]:
class Solution:
    def removeOnes(self, grid):
        return all(grid[i] == grid[0] or grid[i] == [1-val for val in grid[0]] for i in range(len(grid)))

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True


# 6. Evaluate Reverse Polish Notation

Evaluate the value of an arithmetic expression in Reverse Polish Notation.

Valid operators are +, -, *, and /. Each operand may be an integer or another expression.

Note that division between two integers should truncate toward zero.

It is guaranteed that the given RPN expression is always valid. That means the expression would always evaluate to a result, and there will not be any division by zero operation.
```
Example 1:

Input: tokens = ["2","1","+","3","*"]
Output: 9
Explanation: ((2 + 1) * 3) = 9
Example 2:

Input: tokens = ["4","13","5","/","+"]
Output: 6
Explanation: (4 + (13 / 5)) = 6
Example 3:

Input: tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
Output: 22
Explanation: ((10 * (6 / ((9 + 3) * -11))) + 17) + 5
= ((10 * (6 / (12 * -11))) + 17) + 5
= ((10 * (6 / -132)) + 17) + 5
= ((10 * 0) + 17) + 5
= (0 + 17) + 5
= 17 + 5
= 22
```

If you've attempted this question and can't figure out why you're getting wrong answers, here are a couple of things to check before reading the article:

Reverse Polish Notation is not a "reverse" form of Polish Notation. It is a bit different.

If you're using Java, note that the input type is an array of strings, not an array of chars. This means that you should be comparing them with .equals(...), not ==. If your code is working on your computer but not on Leetcode, this is probably why. It is a bug in your code, not in the Leetcode platform.

Some programming languages (e.g. Python, but not C++ and Java) do not truncate towards 0 with division, so you'll need to figure out how to make them do so (we'll discuss ways in the article). For example, if we put -121 // 7 into Python, we get -18, but we actually wanted -17. If unsure about your programming language, either check the documentation or simply write a program that does -121 / 7 (as integer division) and see which result you get.

Did you put numbers around the correct way? e.g. the test case ["12", "7", "-"] means you should calculate 12 - 7 = 5, and not 7 - 12 = -5. In most implementations, getting them the correct way around is not immediately obvious. If you aren't certain you have it right, try this test case (["12", "7", "-]) and check whether you get 5 or -5.

At the time of writing this solution article, the Wikipedia article has a number of errors and sections that are confusing (in particular, parsing the list in reverse). Try to understand how Reverse Polish Notation works and then design an algorithm yourself rather than following the provided pseudocode too closely. The Introduction section of this solution article also explains how Reverse Polish Notation works.

**Introduction**

We'll start with looking at what it means for integer division to truncate towards zero, and then at what Reverse Polish Notation is. 

Division between two integers should truncate towards zero

Early on in your programming career, you probably learned about integer division. When dividing 2 positive numbers, we always truncate down to the nearest integer. The non-integer values are in parenthesis afterwards for reference.
```
6 / 2 = 3 (3.0)
11 / 5 = 2 (2.2)
9 / 5 = 1 (1.8)
```

Most programming languages do integer division by default (as opposed to float division, where decimal places are kept), and this is how they handle positive integers. Both of the following definitions could be (and are) used to describe the truncation.

* The result is truncated to a less than or equal number. i.e. 1 is less than 1.8.
* The truncation is towards zero, i.e. 1 is closer to zero than 1.8 is.

For negative numbers however, it is impossible to satisfy both of these, so one or the other has to be picked. For example, consider the following:
```
-9 / 5 = ? (-1.8)
```

* If we wanted the truncated result to be smaller, we'd have to go to -2, as -2 < -1.
* If we wanted the truncated result to be nearer to zero, we'd have to go to -1 as -1 is nearer to zero than -2 is.
Some programming languages go with the first definition, and others go with the second. For this problem, you are expected to go with the second definition, regardless of what your chosen programming language uses.

Python, for example, goes with the first definition. This means that we need to find a way of doing the division. Luckily, the int(...) function does truncate towards zero, and therefore we can use the int(a / b) trick. Note that this is not the same as int(a // b). We haven't checked what all the programming languages available on Leetcode do, so if your chosen programming language is not truncating division towards zero, have a look in the math libraries for your chosen programming language or do a web search.

**What is Infix Notation?**

Analysing Infix Notation provides some great context for understanding Reverse Polish Notation.

Most people know how to read expressions written using Infix Notation. Evaluating the following expression is something you will have learned to do in elementary school.
```
3 + 1 + 9 - 5 = 8
```
This isn't too difficult. However, many of you will also have seen viral posts circulating social media websites, such as Facebook, that challenge you to evaluate an expression like:
```
5 * 4 + 9 - 2 / 3 + 1 = ?
```
When you check the comments, among many other strange answers, you'll probably see people arguing about whether the answer is 10, 27.33, or 29.33.

The reason for the disagreement is because different people have different understandings about how such an expression should be evaluated.

Those who say the answer is 10 evaluated it strictly from left to right, with the following steps:
```
5 * 4 = 20
20 + 9 = 29
29 - 2 = 27
27 / 3 = 9
9 + 1 = 10
```
Those who say the answer is 27.33 follow a rule where we evaluate operations in the following order; division, multiplication, addition, and subtraction. This method comes from a common misunderstanding of the widely used mnemonics: PEMDAS/BODMAS/BEDMAS. The steps with this method are as follows:
```python
= 5 * 4 + 9 - 0.66 + 1 (Do division first.)
= 20 + 9 - 0.66 + 1 (Do multiplication second.)
= 29 - 1.66 (Do the additions third.)
= 27.33 (Do the subtraction fourth.)
```
Those who say the answer is 29.33 use the rules most programming languages use, and that is also the correct interpretation of the mnemonics (PEMDAS/BODMAS/BEDMAS). That is to do division and multiplication first, in order from left to right, and then addition and subtraction, in order from left to right. Their steps are as follows:
```python
= 20 + 9 - 2 / 3 + 1 (Do the multiplication.)
= 20 + 9 - 0.66 + 1 (Do the division.)
= 29 - 0.66 + 1 (Do the first addition.)
= 28.33 + 1 (Do the subtraction, as it's next.)
= 29.33 (Do the last addition.)
```
Most mathematicians would agree that the correct answer is 29.33. Yet, this is probably not the answer you'd get if you asked a random sample of people in the general public (just look at the Facebook posts!). In school for example, I was taught the method that gives 28.33. It wasn't until I learned programming in university that I learned the correct way!

When we want to do the operations in a different order, we use parenthesis (brackets) around the parts to do first. The parts in parenthesis are always done before the parts outside.

The big disadvantage of Infix Notation is hopefully clear now. The rules for evaluating it are surprisingly complex, cause a lot of confusion, and in fact most people don't understand them properly. Additionally, the need to use parentheses correctly adds another layer of complexity.

As we move towards understanding what Reverse Polish Notation is, keep in mind that while it seems a bit strange and un-intuitive (at first!), that Infix Notation is actually more confusing. The only reason Infix Notation seems intuitive is because you've probably been using it all your life and so it is now second nature to you. People who use Reverse Polish Notation on a daily basis find it very intuitive! Some hand-held calculators still use it!

What is Reverse Polish Notation?

Just like Infix Notation, or in fact any other notation, Reverse Polish Notation has rules for how to evaluate it. You'll need to know these rules before you can write an algorithm. The rules could either be prior knowledge or supplied by an interviewer.
```python
While there are operators remaining in the list, find the left-most operator. Apply it to the 2 numbers immediately before it, and replace all 3 tokens (the operator and 2 numbers) with the result.
```

For example in the most simplest case of 3 4 + when we reach + we can replace 3 4 + with it's result 7.

As long as the input was valid, this rule will always work and leave a single number that should be returned. The leftmost operator that hasn't yet been removed will always have 2 numbers immediately before it.

Hopefully the advantage is obvious now. Reverse Polish Notation doesn't require brackets, and the rules for evaluating it are far simpler. In-fact, our equivalent question for Infix Notation is much more difficult than this one!

**Approach : Evaluate with Stack**

Intuition

We don't want to repeatedly delete items from the middle of a list, as this inevitably leads to O(n^2) time performance. So recall that the above algorithm scanned through the list from left to right, and each time it reached an operator, it'd replace the operator and the 2 numbers immediately before it with the result of applying the operator to the 2 numbers.

The two key steps of the above algorithm were:

* Visit each operator, in linear order. Finding these can be done with a linear search of the original list.
* Get the 2 most recently seen numbers that haven't yet been replaced. These could be tracked using a Stack.

The algorithm would be as follows:
```python
stack = new Stack()
for each token in tokens:
    if token is a number:
        stack.push(token)
    else (token is operator):
        number_2 = stack.pop()  
        number_1 = stack.pop()
        result = apply_operator(token, number_1, number_2)
        stack.push(result)
return stack.pop()
```

ou might have noticed the following 2 lines of the pseudocode could look like they're around the wrong way.
```
number_2 = stack.pop()  
number_1 = stack.pop()
```
They are correct though. Remember that for division and subtraction, the order of the numbers matters. i.e. 7 - 5 ≠ 5 - 7. On the Stack, we have the second on the top. So we need to reverse them before applying the operator.

Algorithm

Here is code that uses lambda functionality.

**Complexity Analysis**

Let nn be the length of the list.

* Time Complexity : O(n)O(n).

We do a linear search to put all numbers on the stack, and process all operators. Processing an operator requires removing 2 numbers off the stack and replacing them with a single number, which is an O(1)O(1) operation. Therefore, the total cost is proportional to the length of the input array. Unlike before, we're no longer doing expensive deletes from the middle of an Array or List.

* Space Complexity : O(n)O(n).

In the worst case, the stack will have all the numbers on it at the same time. This is never more than half the length of the input array.



In [37]:
def evalRPN(tokens):
    # create a stack
    stack = []

    # operations dict
    operations = {
        "+": lambda a, b: a+b,
        "-": lambda a, b: a-b,
        "/": lambda a, b: int(a/b),
        "*": lambda a, b: a*b,
    }

    for token in tokens:
        if token in operations:
            num2 = stack.pop()
            num1 = stack.pop()
            result = operations[token](num1, num2)
            stack.append(result)
        else:
            stack.append(int(token))
    return stack.pop()

# Driver Code
if __name__ == "__main__":
    tokens = ["2", "1", "+", "3", "*"]
    print(evalRPN(tokens))

9


# 7. Minimum Time Difference

Given a list of 24-hour clock time points in "HH:MM" format, return the minimum minutes difference between any two time-points in the list.
```
Example 1:

Input: timePoints = ["23:59","00:00"]
Output: 1
Example 2:

Input: timePoints = ["00:00","23:59","00:00"]
Output: 0
```

Convert each timestamp to it's integer number of minutes past midnight, and sort the array of minutes.
The required minimum difference must be a difference between two adjacent elements in the circular array (so the last element is "adjacent" to the first.) We take the minimum value of all of them.




In [46]:

def findMinDifference(A):
    def convert(time):
        return int(time[:2]) * 60 + int(time[3:])
    minutes = sorted(convert(time) for time in A)
    print(f"minutes = {minutes}")
    mm = minutes[1:] + minutes[:1]
    print(f"mm = {mm}")
    for x, y in zip(minutes, minutes[1:] + minutes[:1]):
        print(x, y)
    return min( (y - x) % (24 * 60) 
                for x, y in zip(minutes, minutes[1:] + minutes[:1]) )

# Driver Code
if __name__ == "__main__":
    A = ["00:00","23:59","00:00"]
    print(findMinDifference(A))

minutes = [0, 0, 1439]
mm = [0, 1439, 0]
0 0
0 1439
1439 0
0


1. Convert each time to minutes. O(n).
2. Sort the list of minutes. O(nlogn).
3. Calculate the distance from p[i] to p[i+1] for all i except p[n-1] where n is the length of the times array. O(n).
4. Calculate the distance of the final point to it's first clockwise element, which will cross 0. This is the point in which there would be a revolution loop. O(1).

At each step between 3 and 4, minDist = min(current, new).

**Time Complexity**
* Time Complexity : O(nlogn)
* Space Complexity : O(1)

In [48]:
import sys
def toMin(time):
    time = time.split(':')
    res = (60*int(time[0])) + int(time[1])
    return res
    
class Solution:
    def findMinDifference(self, timePoints):
        for i, time in enumerate(timePoints):
            timePoints[i] = toMin(time)
        
        res = sys.maxsize
        timePoints.sort()
        for i in range(0, len(timePoints) - 1): #calculate the closest CW distance of each element except last
            res = min(res, (timePoints[i+1] - timePoints[i]))
        
        res = min(res, 60* 24 - timePoints[-1] + timePoints[0]) #calc final point
        
        return res

# Driver Code
if __name__ == "__main__":
    A = ["00:00","23:59","00:00"]
    print(Solution().findMinDifference(A))


0


# 8. Stock Price Fluctuation

You are given a stream of records about a particular stock. Each record contains a timestamp and the corresponding price of the stock at that timestamp.

Unfortunately due to the volatile nature of the stock market, the records do not come in order. Even worse, some records may be incorrect. Another record with the same timestamp may appear later in the stream correcting the price of the previous wrong record.

Design an algorithm that:

* Updates the price of the stock at a particular timestamp, correcting the price from any previous records at the timestamp.
* Finds the latest price of the stock based on the current records. The latest price is the price at the latest timestamp recorded.
* Finds the maximum price the stock has been based on the current records.
* Finds the minimum price the stock has been based on the current records.

Implement the StockPrice class:

* StockPrice() Initializes the object with no price records.
* void update(int timestamp, int price) Updates the price of the stock at the given timestamp.
* int current() Returns the latest price of the stock.
* int maximum() Returns the maximum price of the stock.
* int minimum() Returns the minimum price of the stock.

```
Example 1:

Input
["StockPrice", "update", "update", "current", "maximum", "update", "maximum", "update", "minimum"]
[[], [1, 10], [2, 5], [], [], [1, 3], [], [4, 2], []]
Output
[null, null, null, 5, 10, null, 5, null, 2]

Explanation
StockPrice stockPrice = new StockPrice();
stockPrice.update(1, 10); // Timestamps are [1] with corresponding prices [10].
stockPrice.update(2, 5);  // Timestamps are [1,2] with corresponding prices [10,5].
stockPrice.current();     // return 5, the latest timestamp is 2 with the price being 5.
stockPrice.maximum();     // return 10, the maximum price is 10 at timestamp 1.
stockPrice.update(1, 3);  // The previous timestamp 1 had the wrong price, so it is updated to 3.
                          // Timestamps are [1,2] with corresponding prices [3,5].
stockPrice.maximum();     // return 5, the maximum price is 5 after the correction.
stockPrice.update(4, 2);  // Timestamps are [1,2,4] with corresponding prices [3,5,2].
stockPrice.minimum();     // return 2, the minimum price is 2 at timestamp 4.
```


**Overview**
We are given a stream of records of the price of stock at different timestamps. We need to implement some functions to get the lowest, highest, and latest price of the stock based on the records provided.

The records do not come in chronological order. Another record with the same timestamp may appear later in the stream, correcting the price of the previous wrong record; this correction can affect the lowest, highest, and latest price of the stock.

Let's walk through an example to make sure we clearly understand what this problem is asking. Suppose the stock prices in increasing order of timestamp are, [2, 10, 3, 3, 5, 9].
Here, the lowest price of the stock is 2, the highest price is 10, and the latest price is 9.

If the $1^{st}$ record is corrected from 2 to 4, then the lowest price changes from 2 to 3.

If the last record is corrected from 9 to 6, then the latest price changes to 6.

If $2^{nd}$ record is corrected from 10 to 20, then the highest price changes to 20.

Thus, we need to store all the records of the stock price and make corrections when necessary.
We can use a hashmap to store timestamp as key and price as value, where the correction of records will only take constant time.

Now we need to get the highest and lowest price of the stock. We could think of sorting the stock prices but when an update or an insertion of a new price is made we would need to sort the prices again.

Thus, we will prefer using a data structure that allows us to track the current minimum and maximum values efficiently. **We can use a sorted set, sorted map, or min and max-heaps; the intuition will remain the same only the implementation will differ.** All these data structures keep the elements sorted, and the insertion and removal of elements take only logarithmic time.

**Approach 1: Hashed and Sorted Map**

Intuition

We will use a hashmap (timestampPriceMap) to store the price of the stocks. A hashmap stores the elements as key-value pairs. Here, our key is the timestamp and the value is the price of the stock at the respective timestamp, i.e. the hashmap maps timestamp to price.

Now, we will use a sorted map (priceFrequency) to store the stock prices in increasing order.
A sorted map also stores the elements as key-value pairs and keeps elements sorted on the basis of the key. Thus, we will store the price of the stocks as the key and its occurrence (count) in our records stream as the value, i.e. the sorted map maps the price to the frequency. It denotes how many times a price is present in our records stream.

Insertion of Record:

* Insert the stock price at the current timestamp in timestampPriceMap.
* Increase the stock price's count in priceFrequency. Initially, it is assumed the count is 0 when the price is not present in the map.

Updation of Record:

* Update the stock price of the current timestamp in timestampPriceMap.
* Decrease the count of the old price from priceFrequency and if the count reaches 00 then remove it and increase the correct stock price's count.

Get the Latest Price:

* Use one variable to keep track of the latest time and get the stock's price at the latest time from timestampPriceMap.

Get Minimum and Maximum Stock Price:

* The first element's key of priceFrequency is the lowest price of the stock.
* The last element's key of priceFrequency is the highest price of the stock.

**Algorithm**

* Initialize variables:
    * latestTime, variable to store the latest timestamp according to the records.
    * timestampPriceMap, a hashmap to store timestamp and prices of the stock.
    * priceFrequency, a sorted map to store all prices in increasing order.
* In the update function, the current record can be a new record or a correction to an old record:
    * Try to update latestTime, to the current timestamp.
    * If the current timestamp is already present in timestampPriceMap it means this record is a correction then we reduce the count of the old price from priceFrequency.
    * Add/update the current timestamp's price in timestampPriceMap.
    * Increment the count of the current timestamp's price in priceFrequency.
* In the current function, we need to return the latest price of the stock, i.e. price of the stock at latestTime in timestampPriceMap.
* In the maximum function, we need to return the maximum stock price, i.e. the key of the last element in priceFrequency's.
* In the minimum function, we need to return the minimum stock price, i.e. the key of the first element in priceFrequency's.

**Complexity Analysis**

If NN is the number of records in the input stream.

* Time complexity: O(N \log N)O(NlogN)

    * In the update function, we add and remove a record in both hashmap and sorted map. In hashmap, both operations take constant time, but in the sorted map they take O(\log N)O(logN) time.

    * Each call to the maximum, minimum, or current function will take only constant time to return the result.

    * In the worst-case scenario, all NN calls will be to the update function, which will require a total of O(N \log N)O(NlogN) time.

* Space complexity: O(N)O(N)

    * In the update function, we add and remove a record in both the hashmap and sorted map. Thus each function call takes O(1)O(1) space. So for NN update calls, it will take O(N)O(N) space.

    * The maximum, minimum, and current functions do not use any additional space.

    * Thus, in the worst-case, we will add all NN records in both the hashmap and sorted map, which takes O(N)O(N) space.





In [49]:
from sortedcontainers import SortedDict

class StockPrice:
    def __init__(self):
        self.latest_time = 0
        # Store price of each stock at each timestamp.
        self.timestamp_price_map = {}
        # Store stock prices in increasing order to get min and max price.
        self.price_frequency = SortedDict()
        
    def update(self, timestamp: int, price: int) -> None:
        # Update latest_time to latest timestamp.
        self.latest_time = max(self.latest_time, timestamp)
        
        # If same timestamp occurs again, previous price was wrong. 
        if timestamp in self.timestamp_price_map:
            # Remove previous price.
            old_price = self.timestamp_price_map[timestamp]
            self.price_frequency[old_price] -= 1
            
            # Remove the entry from the sorted-dictionary.
            if not self.price_frequency[old_price]:
                del self.price_frequency[old_price]
        
        # Add latest price for timestamp.
        self.timestamp_price_map[timestamp] = price
        
        if price in self.price_frequency:
            self.price_frequency[price] += 1
        else:
            self.price_frequency[price] = 1

    def current(self) -> int:
        # Return latest price of the stock.
        return self.timestamp_price_map[self.latest_time]
        
    def maximum(self) -> int:
        # Return the maximum price stored at the end of sorted-dictionary.
        return self.price_frequency.peekitem(-1)[0]
        
    def minimum(self) -> int:
        # Return the maximum price stored at the front of sorted-dictionary.
        return self.price_frequency.peekitem(0)[0]

# Driver Code
if __name__ == "__main__":
    stock = StockPrice()
    stock.update(1, 10)
    stock.update(2, 5)
    print(stock.current())
    print(stock.maximum())
    stock.update(1, 3)
    print(stock.maximum())
    stock.update(4, 2)
    print(stock.minimum())
   

5
10
5
2


## Approach 2: Hashmap and Heaps

Intuition

In this approach, again, we will use a hashmap (timestampPriceMap) to record the stock's price at each timestamp.

However, it is not necessary for us to maintain a sorted map as we did in the previous approach. Any time we need to efficiently keep track of the lowest or highest value, we should consider using a heap data structure. Here, we will store each record in 2 different heaps, a min-heap to efficiently track the lowest stock price and a max-heap to efficiently track the highest stock price.

Now the real challenge will be, how to update stock prices?

We can directly change the stock price in the hashmap, but in the heaps, we would have to pop all stock prices until the old price comes on top and then push the new price and all other popped prices back. This would make the update operation very costly.

One way to resolve this issue is every time we get a new price, we push it into each heap, and only while getting the top element we need to verify if the price is correct or outdated.

But how do we know which prices are outdated?

For this, we can use our hashmap (timestampPriceMap). Every time we receive a (price, timestamp) pair, we will set the value for the key (timestamp) to the given price. If the timestamp already exists in the hashmap, we overwrite the old price. So, when finding the maximum or minimum price, we will check to see if the (price, timestamp) pair on the top of the heap agrees with the price listed for the timestamp in the hashmap. If it does not, then the price is outdated, and we discard this pair and get the next top element and again check with the hashmap.

Insertion/Updation of Record:

    * Insert/Update the stock price at the current timestamp in timestampPriceMap.
    * Push the (price, timestamp) pair into the minHeap and maxHeap.

Get the Latest Price:

    * Use one variable to keep track of the latest time and get the stock's price at the latest time from timestampPriceMap.

Get Minimum and Maximum Stock Price:

    * Get the (price, timestamp) pair from the top of minHeap/maxHeap.
    * If timestampPriceMap[timestamp] != price, it means the price for the current timestamp was updated and that this price is outdated. So we discard this pair and repeat the above step. Otherwise, return the current price.

**Algorithm**

* Initialize variables:
    * latestTime, variable to store the latest timestamp according to the records.
    * timestampPriceMap, a hashmap to store timestamp and prices of the stock.
    * minHeap, maxHeap, heaps to store (price, timestamp) pairs and sort elements based on price.
* In the update function, the current record can be a new record or a correction to an old record:
    * Try to update latestTime, to the current timestamp.
    * Add/update the current timestamp's price in timestampPriceMap.
    * Push (price, timestamp) pair in the heaps.
* In the current function, we need to return the latest price of the stock, i.e. price of the stock at latestTime in timestampPriceMap.
* In the maximum / minimum function, we get the (price, timestamp) pair from the top of maxHeap / minHeap. If timestampPriceMap[timestamp] is not same as price, we discard this pair and repeat the same step again. Otherwise, return the current price.

**Implementation**
```callout
Note: In python, in min-heap we push stock prices after multiplying with -1 so that the min-heap behaves as a max-heap. This helps in keeping the implementation simpler.
```

**Complexity Analysis**

Let NN be the number of records in the input stream.

* Time complexity: O(N \log N)O(NlogN)

    * In the update function, we add one record to the hashmap and to each heap. Adding the record to the hashmap takes constant time. However, for a heap, each push operation takes O(\log N)O(logN) time. So for NN update calls, it will take O(N\log N)O(NlogN) worst-case time.

    * Each current function call takes only constant time to return the result.

    * In the maximum and minimum functions, we pop any outdated records that are at the top of the heap. In the worst-case scenario, we might pop (N - 1)(N−1) elements and each pop takes O(\log N)O(logN) time, so it might seem for one function call the time complexity is N \log NNlogN, so for NN functions calls it could be N^{2} \log NN 
    2
    logN. However, when we pop a record from the heap, it's gone and won't be popped again. So overall, if we push NN elements into a heap, we cannot pop more than NN elements, taking into account all function calls. Thus, calls to maximum and minimum will at most require O(N \log N)O(NlogN) time.

* Space complexity: O(N)O(N)

    * In the update function, we add a record to the hashmap and each heap. Since each stock price takes O(1)O(1) space, for NN update calls, it will take O(N)O(N) space.

    * The current function does not use any additional space.

    * In the maximum and minimum functions, we only remove elements from the heap thus these functions also do not use any additional space.

    * Thus, in the worst-case, we will add all NN records to the hashmap and to both heaps, which takes O(N)O(N) space.



In [51]:
from heapq import *
class StockPrice:
    def __init__(self):
        self.latest_time = 0
        # Store price of each stock at each timestamp.
        self.timestamp_price_map = {}
        
        # Store stock prices in sorted order to get min and max price.
        self.max_heap = []
        self.min_heap = []

    def update(self, timestamp: int, price: int) -> None:
        # Update latestTime to latest timestamp.
        self.timestamp_price_map[timestamp] = price
        self.latest_time = max(self.latest_time, timestamp)

        # Add latest price for timestamp.
        heappush(self.min_heap, (price, timestamp))
        heappush(self.max_heap, (-price, timestamp))

    def current(self) -> int:
        # Return latest price of the stock.
        return self.timestamp_price_map[self.latest_time]

    def maximum(self) -> int:
        price, timestamp = self.max_heap[0]

        # Pop pairs from heap with the price doesn't match with hashmap.
        while -price != self.timestamp_price_map[timestamp]:
            heappop(self.max_heap)
            price, timestamp = self.max_heap[0]
            
        return -price

    def minimum(self) -> int:
        price, timestamp = self.min_heap[0]

        # Pop pairs from heap with the price doesn't match with hashmap.
        while price != self.timestamp_price_map[timestamp]:
            heappop(self.min_heap)
            price, timestamp = self.min_heap[0]
            
        return price

# Driver Code
if __name__ == "__main__":
    stock = StockPrice()
    stock.update(1, 10)
    stock.update(2, 5)
    print(stock.current())
    print(stock.maximum())
    stock.update(1, 3)
    print(stock.maximum())
    stock.update(4, 2)
    print(stock.minimum())

5
10
5
2


# 9. Find All Possible Recipes from Given Supplies

You have information about n different recipes. You are given a string array recipes and a 2D string array ingredients. The ith recipe has the name recipes[i], and you can create it if you have all the needed ingredients from ingredients[i]. Ingredients to a recipe may need to be created from other recipes, i.e., ingredients[i] may contain a string that is in recipes.

You are also given a string array supplies containing all the ingredients that you initially have, and you have an infinite supply of all of them.

Return a list of all the recipes that you can create. You may return the answer in any order.

Note that two recipes may contain each other in their ingredients.


```
Example 1:

Input: recipes = ["bread"], ingredients = [["yeast","flour"]], supplies = ["yeast","flour","corn"]
Output: ["bread"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
Example 2:

Input: recipes = ["bread","sandwich"], ingredients = [["yeast","flour"],["bread","meat"]], supplies = ["yeast","flour","meat"]
Output: ["bread","sandwich"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
We can create "sandwich" since we have the ingredient "meat" and can create the ingredient "bread".
Example 3:

Input: recipes = ["bread","sandwich","burger"], ingredients = [["yeast","flour"],["bread","meat"],["sandwich","meat","bread"]], supplies = ["yeast","flour","meat"]
Output: ["bread","sandwich","burger"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
We can create "sandwich" since we have the ingredient "meat" and can create the ingredient "bread".
We can create "burger" since we have the ingredient "meat" and can create the ingredients "bread" and "sandwich".
```

## Algorithm: DFS

We can use a simple DFS; we just need to track can_make for each recipe (undefined, yes or no), so that we traverse each node only once.

To simplify the DFS logic, we check supplies when generating graph, and create a self-loop for recipes without needed supplies. This ensures that those recipes cannot be made.

1. Create a graph/hashmap to store the recipe and its ingredients that are missing from the supplies.
2. The idea is to start with a recipe and check the availability of all the ingredients that the recipe requires, and if any of the ingredients is missing, then repeat the same to see if that missing ingredient(s) can be found within the recipes that may have been prepared already( achieved by calling dfs recursively ). Here, the node is a recipe, and the dictionary graph contains the mapping between a recipe and the corresponding ingredient that is missing from the supplies array.
The can_make dictionary is used to mark if a recipe can be prepared or not.

```
can_make[recipe] = all([dfs(ingr) for ingr in graph[recipe]]) yields True for a recipe that has all the required ingredients in the supplies array, which roughly translates to
can_make[recipe] = all([]) for a recipe say bread that has all the ingredients ['yeast', 'flour'] in the supplies array, which is nothing but the value True.
```


In [55]:
class Solution:
    def findAllRecipes(self, recipes, ingredients, supplies):
        # if ingredient of recipe exist in supplies, then it's good to go
        # if ingredient of recipe doesn't exist in supplies, then use dfs  to check if we can make it from the other ingredients
        
        supplies = set(supplies)
        graph = {}
        res = []

        for recipe, ingredient in zip(recipes, ingredients):
            graph[recipe] = ingredient
        
        visited = set() # to avoid cycles
        def dfs(recipe):
            # if recipe hasn't been visited but it exists in graph, then we can make it
            # if ingredient of recipe doesn't exist and it's already visited or doesn't come from other recipes (graph), then we return False since we don't have enough ingredients
            if recipe in visited:
                return False
            visited.add(recipe)
            if recipe not in graph:
                return False
            for ingredient in graph[recipe]:
                if ingredient not in supplies:
                    if not dfs(ingredient):
                        return False
            res.append(recipe)
            supplies.add(recipe)
            return True
        
        for recipe in recipes:
            dfs(recipe)
        return res
        

# Driver Code
if __name__ == "__main__":
    recipes = ["bread"]
    ingredients = [["yeast","flour"]]
    supplies = ["yeast","flour","corn"]
    print(Solution().findAllRecipes(recipes, ingredients, supplies))


['bread']


# 10. Maximum Number of Points with Cost

You are given an m x n integer matrix points (0-indexed). Starting with 0 points, you want to maximize the number of points you can get from the matrix.

To gain points, you must pick one cell in each row. Picking the cell at coordinates (r, c) will add points[r][c] to your score.

However, you will lose points if you pick a cell too far from the cell that you picked in the previous row. For every two adjacent rows r and r + 1 (where 0 <= r < m - 1), picking cells at coordinates (r, c1) and (r + 1, c2) will subtract abs(c1 - c2) from your score.

Return the maximum number of points you can achieve.

abs(x) is defined as:

x for x >= 0.
-x for x < 0.
 
```
Example 1:
Input: points = [[1,2,3],[1,5,1],[3,1,1]]
Output: 9
Explanation:
The blue cells denote the optimal cells to pick, which have coordinates (0, 2), (1, 1), and (2, 0).
You add 3 + 5 + 3 = 11 to your score.
However, you must subtract abs(2 - 1) + abs(1 - 0) = 2 from your score.
Your final score is 11 - 2 = 9.
```

O(M*N) time and O(N) space


In [57]:
from collections import deque


class Solution:
    def maxPoints(self, points):
        """ DP in Time: O(M*N) Space: O(N)
        
        Define dp[i][j] (i = 0, ..., m - 1; j = 0, ..., n - 1) as max points that one can get from 
        the first i rows (up to the i-th row), and for the i-th row, picking exactly the j-th point
        
        fill dp row by row until the (m - 1)-th row, then return max(dp[-1]) as max points 
        we have initial case: first row dp[0][j] = points[0][j] for j in range(n)
        and for i >= 1: dp[i][j] = points[i][j] + max(dp[i-1][k] - abs(k-j) for k in range(n))
               
        focus on the max term: max(dp[i-1][k] - abs(k-j) for k in range(n)) = max(A[k] + B[k] for k in range(n))
        where A = dp[i-1] remains unchanged over different j, only B shifts right with j
        For example: 
        for each j=0,...,n-1, we need to find the max over A[k] + B[k] to fill dp[i][j]
        A: ..., dp[i-1][j-2], dp[i-1][j-1], dp[i-1][j], dp[i-1][j+1], dp[i-1][j+2], ...
        B: ...,           -2,           -1,          0,           -1,           -2, ...
        
        for j = 0, ..., n-1, 
        there are repetative patterns in this max(A+B) calculation that can be speed up using another DP
        
        for each j
        define left[j] (j=0, ..., n-1) as left[j] = max(A[:j+1] + B[:j+1]) 
        left[0] = max(dp[i-1][0])
        left[1] = max(dp[i-1][0] - 1, dp[i-1][1]) = max(left[0] - 1, dp[i-1][1])
        left[2] = max(dp[i-1][0] - 2, dp[i-1][1] - 1, dp[i-1][2]) = max(left[1] - 1, dp[i-1][2])
        left[3] = max(dp[i-1][0] - 3, dp[i-1][1] - 2, dp[i-1][2] - 1, dp[i-1][3]) = max(left[2] - 1, dp[i-1][3])
        ...
        left can be filled in linear time 
        
        for each j define right[j] (j=0, ..., n-1) as right[j] = max(A[j:] + B[j:])
        right[n-1] = max(dp[i-1][n-1])
        right[n-2] = max(dp[i-1][n-2], dp[i-1][n-1] - 1) = max(dp[i-1][n-2], right[n-1]-1)
        right[n-3] = max(dp[i-1][n-3], dp[i-1][n-2] - 1 , dp[i-1][n-1] - 2) = max(dp[i-1][n-3], right[n-2] - 1)
        ...
        right[0] = max(dp[i-1][0], right[1] - 1)
        right can be filled in linear time
        
        max(left[j], right[j]) 
        = max( 
                max(A[:j+1] + B[:j+1]), 
                max(A[j:]   + B[j:])        
            ) = max(A + B) = max(dp[i-1][k] - abs(k-j) for k in range(n))
            
        thus dp[i][j] = points[i][j] + max(dp[i-1][k] - abs(k-j) for k in range(n)) 
         = points[i][j] + max(left[j], right[j]) 
         can be computed in O(1) time
        """
        num_rows, num_cols = len(points), len(points[0])
        dp_cur, dp_prev = points[0], points[0]
        for i in range(1, num_rows):
            # construct left
            for j in range(num_cols):
                if j == 0:
                    left = [dp_prev[0]]
                else:
                    left.append(max(left[-1] - 1, dp_prev[j]))
            # construct right
            for j in reversed(range(num_cols)):
                if j == num_cols - 1:
                    right = deque([dp_prev[j]])
                else:
                    right.appendleft(max(dp_prev[j], right[0] - 1))
            # fill dp[i][j]
            for j in range(num_cols):
                dp_cur[j] = points[i][j] + max(left[j], right[j])
            dp_prev = dp_cur.copy()
        return max(dp_cur)

# Driver Code
if __name__ == "__main__":
    points = [[1,2,3],[1,5,1],[3,1,1]]
    print(Solution().maxPoints(points))
        


9


# 11. Amount of New Area Painted Each Day

There is a long and thin painting that can be represented by a number line. You are given a 0-indexed 2D integer array paint of length n, where paint[i] = [starti, endi]. This means that on the ith day you need to paint the area between starti and endi.

Painting the same area multiple times will create an uneven painting so you only want to paint each area of the painting at most once.

Return an integer array worklog of length n, where worklog[i] is the amount of new area that you painted on the ith day.

Example 1:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-16-16-diagram-drawio-diagrams-net.png)
```
Input: paint = [[1,4],[4,7],[5,8]]
Output: [3,3,1]
Explanation:
On day 0, paint everything between 1 and 4.
The amount of new area painted on day 0 is 4 - 1 = 3.
On day 1, paint everything between 4 and 7.
The amount of new area painted on day 1 is 7 - 4 = 3.
On day 2, paint everything between 7 and 8.
Everything between 5 and 7 was already painted on day 1.
The amount of new area painted on day 2 is 8 - 7 = 1. 
```
Example 2:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-17-45-diagram-drawio-diagrams-net.png)

```
Input: paint = [[1,4],[5,8],[4,7]]
Output: [3,3,1]
Explanation:
On day 0, paint everything between 1 and 4.
The amount of new area painted on day 0 is 4 - 1 = 3.
On day 1, paint everything between 5 and 8.
The amount of new area painted on day 1 is 8 - 5 = 3.
On day 2, paint everything between 4 and 5.
Everything between 5 and 7 was already painted on day 1.
The amount of new area painted on day 2 is 5 - 4 = 1. 
```
Example 3:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-19-49-diagram-drawio-diagrams-net.png)
```
Input: paint = [[1,5],[2,4]]
Output: [4,0]
Explanation:
On day 0, paint everything between 1 and 5.
The amount of new area painted on day 0 is 5 - 1 = 4.
On day 1, paint nothing because everything between 2 and 4 was already painted on day 0.
The amount of new area painted on day 1 is 0.
```

This is a question testing your data structure skill, the more advanced data structure you know, the easier it will be. I will provide 3 different solutions

- Red-Black Tree (sortedcontainer in python & set in c++)
- Prioirty Queue & Set
- Segment Tree

The basic concept is called sweep line.

For example paint = [[1,4],[5,8],[4,7]], The picture would be like below

![img](https://assets.leetcode.com/users/images/6247c212-6c87-4314-b3e8-f21fee39e3fd_1643862042.2531443.png)

First, put each [start, end] as brown lines, put them on top of the number line one by one ordered by index. and imagine a blue line which will sweep from left to right across all the brown lines.

Moveover, we need another box which can:

- add number
- delete choosen number
- get minimum number

we will discuss the suitable data structure for this box later.

Now, when the blue line sweeps, every time when it touches a start position, it will add the index number to the box, and when it touches an end position, it will remove the index number from the box as the picture shows below.

![img](https://assets.leetcode.com/users/images/e2320b81-40de-4b95-ae06-a57637d4addb_1643998733.061133.png)

Now, the answer is very easy, in each position, you will look at the box and get the minimum number from it. If it has a minimum number, it's the index you should +1 for the answer. Since it means this is the index show up earliest at this position.

## Approach: Using Priority Queue

- The priority queue could add and get minimum
- The set will be used to record those indexes which already ended.

Each time when you want to get minimum number from the priority queue, you need to discard all those ended indexes already recorded in the set.

# Algorithm
1. Go through each index in paint, and add its index to the heap.
2. 


In [61]:
class Solution:
    def amountPainted(self, paint):
        # constructure the sweep line
        records = []
        max_pos = 0
        for i, [start, end] in enumerate(paint):
            records.append((start, i, 1)) # use 1 and -1 to records the type.
            records.append((end, i, -1))
            max_pos = max(max_pos, end)
        records.sort()
        print(f"records is {records}")

        # sweep across all position
        ans = [0 for _ in range(len(paint))]
        indexes = []
        ended_set = set()
        i = 0
        for pos in range(max_pos + 1):
            print(f"Initial Pos is {pos}")
            while i < len(records) and records[i][0] == pos:
                pos, index, type = records[i]
                if type == 1:
                    heapq.heappush(indexes, index)
                else:
                    ended_set.add(index)
                i += 1
            print(f"Indexes after adding to heapq: {indexes}")
            print(f"Ended set: {ended_set}")
            while indexes and indexes[0] in ended_set:
                heapq.heappop(indexes)

            if indexes:
                ans[indexes[0]] += 1
        return ans
# Driver Code
if __name__ == "__main__":
    paint = [[1,4],[5,8],[4,7]]
    print(Solution().amountPainted(paint))

records is [(1, 0, 1), (4, 0, -1), (4, 2, 1), (5, 1, 1), (7, 2, -1), (8, 1, -1)]
Initial Pos is 0
Indexes after adding to heapq: []
Ended set: set()
Initial Pos is 1
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 2
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 3
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 4
Indexes after adding to heapq: [0, 2]
Ended set: {0}
Initial Pos is 5
Indexes after adding to heapq: [1, 2]
Ended set: {0}
Initial Pos is 6
Indexes after adding to heapq: [1, 2]
Ended set: {0}
Initial Pos is 7
Indexes after adding to heapq: [1, 2]
Ended set: {0, 2}
Initial Pos is 8
Indexes after adding to heapq: [1, 2]
Ended set: {0, 1, 2}
[3, 3, 1]


# 12. Snapshot Array

Implement a SnapshotArray that supports the following interface:

* SnapshotArray(int length) initializes an array-like data structure with the given length.  Initially, each element equals 0.
* void set(index, val) sets the element at the given index to be equal to val.
* int snap() takes a snapshot of the array and returns the snap_id: the total number of times we called snap() minus 1.
* int get(index, snap_id) returns the value at the given index, at the time we took the snapshot with the given snap_id
 
```
Example 1:

Input: ["SnapshotArray","set","snap","set","get"]
[[3],[0,5],[],[0,6],[0,0]]
Output: [null,null,0,null,5]
Explanation: 
SnapshotArray snapshotArr = new SnapshotArray(3); // set the length to be 3
snapshotArr.set(0,5);  // Set array[0] = 5
snapshotArr.snap();  // Take a snapshot, return snap_id = 0
snapshotArr.set(0,6);
snapshotArr.get(0,0);  // Get the value of array[0] with snap_id = 0, return 5
```
```
Intuition
Instead of copy the whole array,
we can only record the changes of set.


Explanation
The idea is, the whole array can be large,
and we may take the snap tons of times.
(Like you may always ctrl + S twice)

Instead of record the history of the whole array,
we will record the history of each cell.
And this is the minimum space that we need to record all information.

For each A[i], we will record its history.
With a snap_id and a its value.

When we want to get the value in history, just binary search the time point.


Complexity
Time O(logS)
Space O(S)
where S is the number of set called.

SnapshotArray(int length) is O(N) time
set(int index, int val) is O(1) in Python and O(logSnap) in Java
snap() is O(1)
get(int index, int snap_id) is O(logSnap)
```

In [73]:
import bisect
class SnapshotArray(object):
    
    def __init__(self, length):
        self.d = collections.defaultdict(list)
        self.snap_id = 0

    def set(self, index, val):
        self.d[index].append((self.snap_id, val))
        print(f"self.d is {self.d}")

    def snap(self):
        self.snap_id += 1
        return self.snap_id - 1

    def get(self, index, snap_id):
        i = bisect.bisect(self.d[index], (snap_id, float('inf'))) - 1
        print(f"i is {i}")
        if index in self.d and i != -1:
            return self.d[index][i][1]
        else:
            return 0

# Driver Code
if __name__ == "__main__":
    s = SnapshotArray(3)
    s.set(0, 5)
    snap_id = s.snap()
    print(f"snap_id is {snap_id}")
    s.set(0, 6)
    print(s.get(0, snap_id))


self.d is defaultdict(<class 'list'>, {0: [(0, 5)]})
snap_id is 0
self.d is defaultdict(<class 'list'>, {0: [(0, 5), (1, 6)]})
i is 0
5


# 13. Maximum Split of Positive Even Integers

You are given an integer finalSum. Split it into a sum of a maximum number of unique positive even integers.

For example, given finalSum = 12, the following splits are valid (unique positive even integers summing up to finalSum): (12), (2 + 10), (2 + 4 + 6), and (4 + 8). Among them, (2 + 4 + 6) contains the maximum number of integers. Note that finalSum cannot be split into (2 + 2 + 4 + 4) as all the numbers should be unique.
Return a list of integers that represent a valid split containing a maximum number of integers. If no valid split exists for finalSum, return an empty list. You may return the integers in any order.

 
```
Example 1:

Input: finalSum = 12
Output: [2,4,6]
Explanation: The following are valid splits: (12), (2 + 10), (2 + 4 + 6), and (4 + 8).
(2 + 4 + 6) has the maximum number of integers, which is 3. Thus, we return [2,4,6].
Note that [2,6,4], [6,2,4], etc. are also accepted.
Example 2:

Input: finalSum = 7
Output: []
Explanation: There are no valid splits for the given finalSum.
Thus, we return an empty array.
Example 3:

Input: finalSum = 28
Output: [6,8,2,12]
Explanation: The following are valid splits: (2 + 26), (6 + 8 + 2 + 12), and (4 + 24). 
(6 + 8 + 2 + 12) has the maximum number of integers, which is 4. Thus, we return [6,8,2,12].
Note that [10,2,4,12], [6,2,4,16], etc. are also accepted.
```

## Approach:
```
So it's clear from question that if n is odd answer is not possible (bcz we can't represent a odd number as a sum of even numbers)

Now if n is even , then we have to make the largest list of unique even number such that the sum is equal to given n.

Now to make the largest we have to take smallest numbers first like 2,4,6,8... and so on.

but wait what happen if we are doing in this manner and the total sum is greater than desired , no worries , we wll do this step untill our sum is less than or equal to given number , and just add the remaining difference to last number in the list.

Take n =14

i = 2 , crSum = 0 , list = [] (crSum + 2 <= 14 , so push it) , crSum + i = 2 , list = [2]
i = 4 , crSum = 2 , list = [2] (crSum + 4 <= 14 , so push it) , crSum + i = 6 , list = [2,4]
i = 6 , crSum = 6 , list = [2,4] (crSum + 6 <= 14 , so push it) , crSum + i = 12 , list = [2,4,6]
i = 8 , crSum = 12 , list = [2,4,6] (crSum + 8 > 14 , so don't push it , break the loop)
Now we have crSum = 12 , and we want 14 , so simply add difference (which is 14-12 = 2 ) in the last element of list

so list = [2,4,6+(14-12)] = [2,4,8]
```

In [85]:
class Solution:
    def maximumEvenSplit(self, finalSum):
        if finalSum % 2 != 0:
            return 0
        num = 2
        temp_sum = 0
        list_sum = []
        while temp_sum + num <= finalSum:
            list_sum.append(num)
            temp_sum += num
            num += 2
            
            print(f"temp_sum is {temp_sum}")
        # add the difference between finalSum and temp_sum to the last element of list_sum
        
        list_sum[-1] += finalSum - temp_sum
        return list_sum 

# Driver Code
if __name__ == "__main__":
    print(Solution().maximumEvenSplit(10))

temp_sum is 2
temp_sum is 6
[2, 8]


# 14. Logger Rate Limiter

Design a logger system that receives a stream of messages along with their timestamps. Each unique message should only be printed at most every 10 seconds (i.e. a message printed at timestamp t will prevent other identical messages from being printed until timestamp t + 10).

All messages will come in chronological order. Several messages may arrive at the same timestamp.

Implement the Logger class:

Logger() Initializes the logger object.
bool shouldPrintMessage(int timestamp, string message) Returns true if the message should be printed in the given timestamp, otherwise returns false.
 
```
Example 1:

Input
["Logger", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage"]
[[], [1, "foo"], [2, "bar"], [3, "foo"], [8, "bar"], [10, "foo"], [11, "foo"]]
Output
[null, true, true, false, false, false, true]

Explanation
Logger logger = new Logger();
logger.shouldPrintMessage(1, "foo");  // return true, next allowed timestamp for "foo" is 1 + 10 = 11
logger.shouldPrintMessage(2, "bar");  // return true, next allowed timestamp for "bar" is 2 + 10 = 12
logger.shouldPrintMessage(3, "foo");  // 3 < 11, return false
logger.shouldPrintMessage(8, "bar");  // 8 < 12, return false
logger.shouldPrintMessage(10, "foo"); // 10 < 11, return false
logger.shouldPrintMessage(11, "foo"); // 11 >= 11, return true, next allowed timestamp for "foo" is 11 + 10 = 21
```

Approach : Hashtable / Dictionary

Intuition

One could combine the queue and set data structure into a hashtable or dictionary, which gives us the capacity of keeping all unique messages as of queue as well as the capacity to quickly evaluate the duplication of messages as of set.

The idea is that we keep a hashtable/dictionary with the message as key, and its timestamp as the value. The hashtable keeps all the unique messages along with the latest timestamp that the message was printed.
![img](https://leetcode.com/problems/logger-rate-limiter/Figures/359/359_hashtable.png)

As one can see from the above example, there is an entry in the hashtable with the message m2 and the timestamp 2. Then there comes another message m2 with the timestamp 15. Since the message was printed 13 seconds before (i.e. beyond the buffer window), it is therefore eligible to print again the message. As a result, the timestamp of the message m2 would be updated to 15.

Algorithm

* We initialize a hashtable/dictionary to keep the messages along with the timestamp.

* At the arrival of a new message, the message is eligible to be printed with either of the two conditions as follows:

    * case 1). we have never seen the message before.

    * case 2). we have seen the message before, and it was printed more than 10 seconds ago.

* In both of the above cases, we would then update the entry that is associated with the message in the hashtable, with the latest timestamp.

Complexity Analysis

Time Complexity: \mathcal{O}(1)O(1). The lookup and update of the hashtable takes a constant time.

Space Complexity: \mathcal{O}(M)O(M) where MM is the size of all incoming messages. Over the time, the hashtable would have an entry for each unique message that has appeared.


In [86]:
class Logger:
    
    def __init__(self):
        self.dictionary = {}
        

    def shouldPrintMessage(self, timestamp: int, message: str) -> bool:
        if message not in self.dictionary or self.dictionary[message] + 10 <= timestamp:
            self.dictionary[message] = timestamp
            return True
        else:
            return False

# Driver Code
if __name__ == "__main__":
    l = Logger()
    print(l.shouldPrintMessage(1,"foo"))
    print(l.shouldPrintMessage(2,"bar"))
    print(l.shouldPrintMessage(3,"foo"))
    print(l.shouldPrintMessage(8,"bar"))
    print(l.shouldPrintMessage(10,"foo"))
    print(l.shouldPrintMessage(11,"foo"))


True
True
False
False
False
True
